# Concurrency: creating multi-threaded programs
_COSC 208, Introduction to Computer Systems, Spring 2025_

## Creating many threads

* Create an array of `pthread_t` and an array of arguments
* Call `pthread_create` within a loop
* Call `pthread_join` within a separate loop

_Example_

In [36]:
/* 1*/  #include <pthread.h>
/* 2*/  #include <stdio.h>
/* 3*/  #include <stdlib.h>
/* 4*/  #define NUM_THREADS 5
/* 5*/  void *simple(void *arg) {
/* 6*/      int *id = (int *)arg;
/* 7*/      printf("I am thread %d\n", *id);
/* 8*/      return NULL;
/* 9*/  }
/*10*/  int main() {
/*11*/      pthread_t threads[NUM_THREADS];
/*12*/      int ids[NUM_THREADS];
/*13*/      for (int i = 0; i < NUM_THREADS; i++) {
/*14*/          ids[i] = i+1;
/*15*/          pthread_create(threads+i, NULL, &simple, ids+i);
/*16*/      } 
/*17*/      for (int i = 0; i < NUM_THREADS; i++) {
/*18*/          pthread_join(threads[i], NULL);
/*19*/      }
/*20*/      printf("All threads finished\n");
/*21*/  }

I am thread 1
I am thread 2
I am thread 3
I am thread 4
I am thread 5
All threads finished


## Returning values from threads

* _When does a thread end?_ — when the function passed to `pthread_create` finishes (i.e., returns)
* _What happens to a function's parameters and local variables when the function returns?_ — they no longer exist (i.e., the stack frame is destroyed)
* _Where should we store a value that should exist even after a function returns?_ — on the heap
* Need to store a thread's return value on the heap
* Thread returns a pointer to the value on the heap

_Example_

In [20]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <string.h>
/* 4*/  #include <pthread.h>
/* 5*/  void *length(void *arg) {
/* 6*/      char *str = (char *)arg;
/* 7*/      int *len = malloc(sizeof(int));
/* 8*/      *len = strlen(str);
/* 9*/      return len;
/*10*/  }
/*11*/  int main() {
/*12*/      pthread_t thread;
/*13*/      char *phrase = "Hello, threads!";
/*14*/      pthread_create(&thread, NULL, &length, phrase);
/*15*/      int *result = NULL;
/*16*/      pthread_join(thread, (void **)&result);
/*17*/      printf("Length: %d\n", *result);
/*18*/      free(result);
/*19*/  }

Length: 15


* `pthread_join` returns `0` if successful, or an error number
* To get the pointer returned by the thread, we need to pass a location where the pointer can be stored — i.e., we need to pass a double pointer

## Writing multi-threaded programs

Q1: _Write a function called `sum_array` which takes an array of `ARRAY_LEN` integers and returns the sum of the integers. Your function should have the appropriate prototype/implementation to serve as the entry point for a thread. Assume `ARRAY_LEN` is a constant which has been `#define`d._

<p style="height:25em;"></p>

Q2: _Write a function called `sum_matrix` which takes an array of `NUM_ARRAYS` arrays of integers (i.e., an `int **`) and returns the sum of all the integers. The function should create `NUM_ARRAYS` threads, each running the `sum_array` function for a single array of integers. Assume `NUM_ARRAYS` is a constant which has been `#define`d._

In [2]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

#define ARRAY_LEN 10
#define NUM_ARRAYS 5

void *sum_array(void *args) {
    int *nums = (int *)args;
    int *sum = malloc(sizeof(int));
    *sum = 0;
    for (int i = 0; i < ARRAY_LEN; i++) {
        *sum += nums[i];
    }
    return sum;
}

int sum_matrix(int *matrix[]) {
    pthread_t threads[NUM_ARRAYS];
    for (int i = 0; i < NUM_ARRAYS; i++) {
        pthread_create(&(threads[i]), NULL, &sum_array, matrix[i]);
    }

    int total = 0;
    for (int i = 0; i < NUM_ARRAYS; i++) {
        int *sum;
        pthread_join(threads[i], (void **)(&sum));
        total += *sum;
        free(sum);
    }

    return total;
}

int main() {
    int *matrix[NUM_ARRAYS];
    for (int i = 0; i < NUM_ARRAYS; i++) {
        matrix[i] = malloc(sizeof(int) * ARRAY_LEN);
        for (int j = 0; j < ARRAY_LEN; j++) {
            matrix[i][j] = i * 100 + j;
        }
    }

    int sum = sum_matrix(matrix);
    printf("%d\n", sum);
}

10225


<p style="height:25em;"></p>

Q3: _The following partially written program takes an integer (`len`) and a string (`word`) as command-line arguments and creates a thread that calls the `trim` function. Finish writing the code such that the `trim` function returns the first `len` characters of `word`, and the main function prints the returned string after the thread completes._

In [6]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

struct trim_args {
    int len;
    char *word;
};

void *trim(void *arg) {
    struct trim_args *targs = (struct trim_args *)arg;
    char *result = malloc(sizeof(char) * (targs->len + 1));
    for (int i = 0; i < targs->len; i++) {
        result[i] = targs->word[i];
    }
    result[targs->len] = '\0';
    return result;
}

int main(int argc, char *argv[]) {
    // Dummy command-line arguments
    argc = 3;
    argv = malloc(sizeof(char *) * 3);
    argv[1] = "3";
    argv[2] = "December";
    
    if (argc != 3) {
        printf("Provide an integer and a word\n");
        return 1;
    }
    int len = atoi(argv[1]);
    char *word = argv[2];
    
    struct trim_args targs = {len, word};
    
    pthread_t thr;
    pthread_create(&thr, NULL, &trim, &targs);
    
    char *result = NULL;
    pthread_join(thr, (void**) &result);
    printf("%s\n", result);
}

Dec


In [ ]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>








void *trim(void *arg) {

    
    
    
    


    

    
    
    
    
    
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        printf("Provide an integer and a word\n");
        return 1;
    }
    int len = atoi(argv[1]);
    char *word = argv[2];
    

    
    
    pthread_t thr;
    pthread_create(&thr, NULL, &trim, 


        
                   
                   
                   

}

## Passing multiple parameters to threads

* A thread can only take a single `void *` pointer as a parameter
* To pass multiple parameters:
    * Define a `struct` that contains a field for each of the parameters
    * Allocate space for an instance of the `struct`
    * Pass a pointer to the instance to the thread — i.e., as the last parameter in `pthread_create`

## Transforming programs to be multi-threaded

* For-each style loops are natural candidates for threading
    * Example: a program that counts the number of occurrences of each vowel within a string
    * Another example: a program that counts the longest sequence of each Short Tandem Repeat (STR) in a DNA sequence
* Steps
    * Move the body of the loop into its own "worker" function (if not done already)
    * Write a "wrapper" function which:
        * Has the required signature for a thread function (i.e., takes a single `void *` parameter and returns a void pointer)
        * Extracts the argument(s) for the "worker" function from the thread function's arguments
        * Calls the "worker" function
        * Prepares the return value from the "worker" function to be returned by the thread
    * Replace the body of the loop with the creation of threads that execute the "wrapper" function
    * Add a second loop to wait for all threads to complete

_Assume you are given the following code:_

In [ ]:
#include <ctype.h>
#include <stdio.h>
#include <string.h>
int count(char *str, char ch) {
    int num = 0;
    while (*str != '\0') {
        if (tolower(*str) == ch) {
            num++;
        }
        str++;
    }
    return num;
}
int main(int argc, char *argv[]) {
    if (argc < 2) {
        printf("Error: provide a string\n");
        return 1;
    }
    char *str = argv[1];
    char *vowels = "aeiou";
    int counts[strlen(vowels)];
    for (int i = 0; i < strlen(vowels); i++) {
        counts[i] = count(str, vowels[i]);
    }

    for (int i = 0; i < strlen(vowels); i++) {
        printf("%c %d\n", vowels[i], counts[i]);
    }
}

Q4: _Write a function called  `count_wrapper` that has the signature required for a thread function and calls the `count` function. (Hint: you'll need to declare a `struct` that contains all of the parameters required for `count`.)_

<p style="height:20em;"></p>

Q5: _Re-write `main` to create/wait for threads that execute `count_wrapper` (instead of calling `count` sequentially)._

In [ ]:
#include <ctype.h>
#include <stdio.h>
#include <string.h>
#include <pthread.h>
int count(char *str, char ch) {
    int num = 0;
    while (*str != '\0') {
        if (tolower(*str) == ch) {
            num++;
        }
        str++;
    }
    return num;
}
struct count_args {
    char *str;
    char ch;
};
void *count_wrapper(void *arg) {
    struct count_args *cargs = (struct count_args *)arg;
    int *result = malloc(sizeof(int));
    *result = count(cargs->str, cargs->ch);
    return result;
}
int main(int argc, char *argv[]) {
    if (argc < 2) {
        printf("Error: provide a string\n");
        return 1;
    }
    char *str = argv[1];
    char *vowels = "aeiou";

    // Create threads
    struct count_args args[strlen(vowels)];
    pthread_t threads[strlen(vowels)];
    for (int i = 0; i < strlen(vowels); i++) {
        args[i].str = str;
        args[i].ch = vowels[i];
        pthread_create(&(threads[i]), NULL, &count_wrapper, &(args[i]));
    }

    // Wait for threads
    for (int i = 0; i < strlen(vowels); i++) {
        int *ptr;
        pthread_join(threads[i], (void **)(&ptr));
        printf("%c %d\n", vowels[i], *ptr);
    }
}